In [12]:
# knn, SVM, random forest
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas
print("starting")
df = pandas.read_csv("hourly.csv", sep=',', error_bad_lines=False) #ignore bad lines
print(df.columns)
print(df.shape)



starting
Index([u'URL', u'Keywords', u'Keyword Count', u'DateTime', u'Favorite Count',
       u'Retweet', u'Lang', u'LinkCount', u'Link1', u'Link2', u'Link3',
       u'Author', u'Text', u'Followers', u'Friends', u'Location', u'Timezone',
       u'UTC Offset'],
      dtype='object')
(60000, 18)


In [14]:
df = df[pandas.notnull(df['Text'])]
print(df.columns)
print(df.shape)

Index([u'URL', u'Keywords', u'Keyword Count', u'DateTime', u'Favorite Count',
       u'Retweet', u'Lang', u'LinkCount', u'Link1', u'Link2', u'Link3',
       u'Author', u'Text', u'Followers', u'Friends', u'Location', u'Timezone',
       u'UTC Offset'],
      dtype='object')
(59983, 18)


In [15]:
# set threshold for binary classifier
#retweets = df.Retweet
threshold = 5000
# train_labels = []
# for num in retweets:
#     if num >= threshold:
#         train_labels.append(1) #viral
#     else:
#         train_labels.append(0) #not viral
df['Viral'] =df['Retweet'].apply(lambda retweet: 1 if retweet >= threshold else 0)
train_labels = df.Viral.values
labels = list(set(train_labels))




In [65]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.stem.snowball import *
stemmer = SnowballStemmer('english')
#stemmer = PorterStemmer('english')

#stopwords?
#stop = ['amp', 'cc', 'did', 'don', 'rt', 'll', 'oh', 've', 'yes', 'let', 'going', 'via', 're', 'tweet' ]
stop = []
#http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
# preProcess(str):
#     url_pattern = re.compile(r'http(s?)://[\w./]+')
#     pic_pattern = re.compile(r'pic.twitter.com/[\w.]+')
#     str = pic_pattern.sub("", str)
#     str = url_pattern.sub("", str)
#     return str
# http://shahmirj.com/blog/extracting-twitter-usertags-using-regex
class NoUrls_CountVectorizer(CountVectorizer):
    def build_preprocessor(self):
        url_pattern = re.compile(r'http(s?)://[\w./]+')
        pic_pattern = re.compile(r'pic.twitter.com/[\w.]+')
        preprocessor = super(NoUrls_CountVectorizer, self).build_preprocessor()
        return lambda doc: (pic_pattern.sub('', url_pattern.sub('', preprocessor(doc)) ))

class NoUrls_Stemmed_CountVectorizer(CountVectorizer):
    def build_preprocessor(self):
        url_pattern = re.compile(r'(?:\@|https?://)\S+')
        #url_pattern = re.compile(r'http(s?)://)[\w./]+')
        pic_pattern = re.compile(r'pic.twitter.com/[\w.]+')
        at_pattern = re.compile(r' (?<=^|(?<=[^a-zA-Z0-9-_\\.]))@([A-Za-z]+[A-Za-z0-9_]+)')
        preprocessor = super(NoUrls_Stemmed_CountVectorizer, self).build_preprocessor()
        return lambda doc: (pic_pattern.sub('', url_pattern.sub('', preprocessor(doc)) ))
   #right now just doing splits on whitespace and stemming 
    def build_tokenizer(self):
        tokenizer = super(NoUrls_Stemmed_CountVectorizer, self).build_tokenizer()
        return lambda doc: (stemmer.stem(w) for w in tokenizer(doc))

#bigrams? ngram_range=(1,2)
vectorizer = NoUrls_Stemmed_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')
   
#example processing
# tweet = "RT @femaIes: When you gotta go visit your broads in Atlanta https://t.co/pCau23tv3q"
# tweet = unicode(tweet, "ascii", "ignore")
# print tweet
# print 'Preprocess:', vectorizer.build_preprocessor()(tweet)
# print
# print 'Analyze:', vectorizer.build_analyzer()(tweet)
# vectorizer.fit_transform([tweet])
# vectorizer.vocabulary_  

print "done"

done


In [33]:
documents = []
for doc in df.Text:
    doc = doc.decode("utf-8", "ignore")
    documents.append(doc)

df.Text = documents

In [28]:
#documents = df.Text
X_count = vectorizer.fit_transform(documents)
#print X_count
#print X_count.toarray()
print vectorizer.get_feature_names()


[u'00', u'000', u'0000', u'00000', u'00000049', u'0000005', u'00000335', u'00003346', u'00003840', u'00004004', u'00004332', u'00004948', u'00004985', u'00005103', u'00005112', u'0000hrs', u'0006', u'000hpa', u'000in', u'000won', u'001', u'0014', u'002', u'0022', u'003', u'00348736', u'00351500', u'004', u'004in', u'005', u'0054', u'005in', u'006', u'00633', u'006in', u'007', u'0070', u'008', u'0083', u'0086', u'009', u'00a', u'00am', u'00buy', u'00c', u'00h', u'00h00', u'00hpa', u'00in', u'00mm', u'00mst', u'00p', u'00ph', u'00pm', u'00purchas', u'00s', u'00utc', u'00z', u'01', u'0100', u'01001', u'0103', u'0104554', u'010in', u'011', u'0113', u'01132559555', u'0115', u'012', u'01223', u'01253', u'01253625262', u'01273', u'012in', u'013', u'013in', u'013ss', u'01412266000', u'014in', u'0151', u'01510', u'01591', u'015in', u'016', u'0161', u'01672b40', u'017', u'01788', u'01803', u'01845', u'018in', u'01978', u'01_12', u'01am', u'01hpa', u'01in', u'01line', u'01mb', u'01molleto', u'01p

In [29]:
#http://scikit-learn.org/stable/auto_examples/feature_stacker.html
#trying to append to feature list X
#http://stackoverflow.com/questions/19466868/how-do-i-do-classification-using-tfidfvectorizer-plus-metadata-in-practice
import scipy
linkCount = [df.LinkCount]
linkCount = np.array(linkCount)
X_metadata = scipy.sparse.csr_matrix(linkCount.T) #must transpose to get shape (___, 1)
#print X_metadata
#print X_count
#X_metadata = [df.LinkCount.as_matrix()]
print X_count.shape
print X_metadata.shape
X = scipy.sparse.hstack([X_count, X_metadata])
#print X.toarray()

print X.shape

(59983, 42589)
(59983, 1)
(59983, 42590)


In [30]:
train_labels = np.array(train_labels)
train_features = X

print "train labels: "
print train_labels
print labels
print 
print "train features:"
#print train_features.toarray()

train labels: 
[0 0 0 ..., 0 0 0]
[0, 1]

train features:


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

chi2score = chi2(train_features, train_labels)[0]

from pylab import barh,plot,yticks,show,grid,xlabel,figure
figure(figsize=(10,20))
wscores = zip(vectorizer.get_feature_names(), chi2score)
wchi2 = sorted(wscores, key=lambda x:x[1]) 
topchi2 = zip(*wchi2[-100:])
x = range(len(topchi2[1]))
labels = topchi2[0]
barh(x,topchi2[1],align='center',alpha=.2,color='g')
plot(topchi2[1],x,'-o',markersize=2,alpha=.8,color='g')
yticks(x,labels)
xlabel('$\chi^2$')
show()

In [31]:
#Make SVM classifier
from sklearn import svm
#linear or not?
name = "Liblinear"
#loss=?, penalty=?, dual=False?, tol=1e-3
classifier = svm.LinearSVC()
classifier.fit(train_features, train_labels)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [58]:
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report

'''helper function for displaying best found features on grid_search'''
def print_grid_search_metrics(gs):
    print("Best score: %0.3f" % gs.best_score_)
    print("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
'''helper function for displaying the algorithm metrics'''
def print_metrics(model_name, y_labels, y_predicted):
    
    print "MODEL: " + model_name
    print 'Test Accuracy: ' + str(metrics.accuracy_score(y_labels, y_predicted))
    
    print '\nClassification report:'
    print classification_report(y_labels, y_predicted, target_names=['non-fashion tweets', 'fashion tweets'])
    
    print '\nConfusion matrix:'
    print metrics.confusion_matrix(y_labels, y_predicted)
    
'''helper to display the most informative features for each group'''
def show_most_informative_features(vectorizer, clf, n=20):
    feature_names = vectorizer.get_feature_names()
    coefs_with_names = sorted(zip(clf.coef_[0], feature_names))
    top_features = zip(coefs_with_names[:n], coefs_with_names[:-(n + 1):-1])  #top features for both groups
    for (coef_1, fn_1), (coef_2, fn_2) in top_features:
        print "\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2)

In [34]:
#Make a pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',   NoUrls_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')),
    ('classifier',  svm.LinearSVC()) ])

from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
#adapted from http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
k_fold = KFold(n=len(df), n_folds=10)
scores = []
confusion = np.array([[0, 0], [0, 0]])

for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['Text'].values
    train_y = df.iloc[train_indices]['Viral'].values
    
    test_text = df.iloc[test_indices]['Text'].values
    test_y = df.iloc[test_indices]['Viral'].values
    
    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #update totals
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)
print('Total tweets classified:', len(df))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

('Total tweets classified:', 59983)
('Score:', 0.78289460167418568)
Confusion matrix:
[[56926   419]
 [  670  1968]]


In [ ]:
#Make kNN classifier
#NearestCentroid, KNeighborsClassifier
from sklearn import neighbors
name = "kNN"
# classifier = neighbors.
# KNeighborsClassifier(n_neighbors = 10)
# classifier.fit(train_features, train_labels)

#Make a pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',   NoUrls_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')),
    ('classifier',  neighbors.KNeighborsClassifier(n_neighbors = 4)) ])

from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
#adapted from http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
k_fold = KFold(n=len(df), n_folds=10)
scores = []
confusion = np.array([[0, 0], [0, 0]])

for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['Text'].values
    train_y = df.iloc[train_indices]['Viral'].values
    
    test_text = df.iloc[test_indices]['Text'].values
    test_y = df.iloc[test_indices]['Viral'].values
    
    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #update totals
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)
print('Total tweets classified:', len(df))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

('Total tweets classified:', 59983)
('Score:', 0.64278182480655954)
Confusion matrix:
[[57157   188]
 [ 1299  1339]]


In [ ]:
#Make Random Forest clasisifier
from sklearn import ensemble
# name = "randomforest"
# classifier = ensemble.RandomForestClassifier(n_estimators=100)
# classifier.fit(train_features, train_labels)

#Make a pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',   NoUrls_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')),
    ('classifier',  ensemble.RandomForestClassifier(n_estimators=100)) ])

from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
#adapted from http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
k_fold = KFold(n=len(df), n_folds=10)
scores = []
confusion = np.array([[0, 0], [0, 0]])

for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['Text'].values
    train_y = df.iloc[train_indices]['Viral'].values
    
    test_text = df.iloc[test_indices]['Text'].values
    test_y = df.iloc[test_indices]['Viral'].values
    
    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #update totals
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)
print('Total tweets classified:', len(df))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

In [56]:
#Make Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
# name = "randomforest"
# classifier = ensemble.RandomForestClassifier(n_estimators=100)
# classifier.fit(train_features, train_labels)

#Make a pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',   NoUrls_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')),
    ('classifier',  LogisticRegression()) ])

from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
#adapted from http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
k_fold = KFold(n=len(df), n_folds=10)
scores = []
confusion = np.array([[0, 0], [0, 0]])

for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['Text'].values
    train_y = df.iloc[train_indices]['Viral'].values
    
    test_text = df.iloc[test_indices]['Text'].values
    test_y = df.iloc[test_indices]['Viral'].values
    
    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #update totals
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)
print('Total tweets classified:', len(df))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

('Total tweets classified:', 59983)
('Score:', 0.71101369903241918)
Confusion matrix:
[[57224   121]
 [ 1115  1523]]


In [57]:
#Make Decision Tree Classifier
from sklearn import tree
# name = "randomforest"
# classifier = ensemble.RandomForestClassifier(n_estimators=100)
# classifier.fit(train_features, train_labels)

#Make a pipeline
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('vectorizer',   NoUrls_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')),
    ('classifier',  tree.DecisionTreeClassifier()) ])

from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
#adapted from http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
k_fold = KFold(n=len(df), n_folds=10)
scores = []
confusion = np.array([[0, 0], [0, 0]])

for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['Text'].values
    train_y = df.iloc[train_indices]['Viral'].values
    
    test_text = df.iloc[test_indices]['Text'].values
    test_y = df.iloc[test_indices]['Viral'].values
    
    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #update totals
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)
print('Total tweets classified:', len(df))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

('Total tweets classified:', 59983)
('Score:', 0.80079986967723416)
Confusion matrix:
[[56934   411]
 [  600  2038]]


In [66]:
vectorizer = NoUrls_CountVectorizer(ngram_range = (1,1), binary =True, min_df=1, stop_words=stop, strip_accents='ascii')
clf = svm.LinearSVC()
pipeline = Pipeline([('vectorizer', vectorizer), ('classifier', clf)])
predictive_model = pipeline.fit(df['Text'].values, df['Viral'].values)
show_most_informative_features(vectorizer, clf, 20)

	-1.8413	palette        		2.4343	tweetlikeagiri 
	-1.3297	whirl          		2.3404	shanyia_brown  
	-1.1950	belong         		2.0298	isweatergawd   
	-1.1742	liamhemsworth  		1.9641	girlposts      
	-1.1692	aching         		1.9608	camerondallas  
	-1.1618	tells          		1.9046	snoozeanne     
	-1.1579	geniusfootball 		1.8655	ashley_emiko   
	-1.1074	crazyfightz    		1.8571	rt             
	-1.1021	alyricz        		1.8123	niallofficial  
	-1.0841	tigerbeatnow   		1.8065	nike           
	-1.0832	girlfriend     		1.7639	juliancamarena 
	-1.0766	pupper         		1.7581	nohoesnextdoor 
	-1.0312	fmuswazz       		1.7434	myiostsoul     
	-1.0227	society        		1.7370	bivckprincess  
	-1.0216	styie          		1.7296	jypnation      
	-0.9971	tupac          		1.6863	metrroboomin   
	-0.9936	horizonjuicy   		1.6726	shvrdae        
	-0.9910	auntie         		1.6492	comedyorjoke   
	-0.9851	finalsweek     		1.6474	temidayoooo    
	-0.9844	biebshim       		1.6352	minaj_gabby    


In [52]:
from sklearn.base import TransformerMixin
class DenseTransformer(TransformerMixin):

    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self

In [54]:
# NOTE: code is correct, but memory error
# name = "naivebayes"

#Make a pipeline
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
pipeline = Pipeline([
    ('vectorizer',   NoUrls_CountVectorizer(ngram_range = (1,1), binary =True,
                                    min_df=1, stop_words=stop, strip_accents='ascii')),
    ('to_dense', DenseTransformer()),
    ('classifier',  clf) ])
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
#adapted from http://zacstewart.com/2015/04/28/document-classification-with-scikit-learn.html
k_fold = KFold(n=len(df), n_folds=10)
scores = []
confusion = np.array([[0, 0], [0, 0]])

for train_indices, test_indices in k_fold:
    train_text = df.iloc[train_indices]['Text'].values
    train_y = df.iloc[train_indices]['Viral'].values
    
    test_text = df.iloc[test_indices]['Text'].values
    test_y = df.iloc[test_indices]['Viral'].values
    
    pipeline.fit(train_text, train_y)
    predictions = pipeline.predict(test_text)

    #update totals
    confusion += confusion_matrix(test_y, predictions)
    score = f1_score(test_y, predictions, pos_label=1)
    scores.append(score)
print('Total tweets classified:', len(df))
print('Score:', sum(scores)/len(scores))
print('Confusion matrix:')
print(confusion)

MemoryError: 

[ u"RT @THSWxSTEM: It's windy at Tolland High School! The wind speed is a brisk 16 miles per hour."
 u"I thought I'd be able to go to sleep now ??"
 u"RT @davidwalliams: #Prince with Tom Petty, Steve Winwood &amp; Jeff Lynne performing George Harrison's 'While My Guitar Gently Weeps' https://t"
 ..., u'https://t.co/wl9YbaIJA7 from https://t.co/xK1fRO7ym0 #????'
 u'RT @MyAceTv: Rihanna flaunts her enviable long legs in elegant hosiery. See her latest fashion https://t.co/x7rM0XALRz https://t.co/E5Yoi0'
 u"RT @lynlinking: Malcolm Turnbull's phony thoroughly liberal leadership  by John Roskam executive director Institute of Public   https://t.c"]
